In [33]:

import random
UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3

class GridworldEnv():
  def __init__(self):
    self.shape = [4,4]
    self.state_space = 16
    self.action_space = 4


  def next_state_probabilities(self):
    next_state_probs = []
    for state_number in range(self.state_space):
      state_prob = []
      if(state_number == 0 or state_number == 15):
        done = True
        reward = 0
        for action_pos in range(self.action_space):
          state_prob.append((1,state_number, reward, done))
        next_state_probs.append(state_prob)
      else:
        for action_pos in range(self.action_space):
          done = False
          reward = -1
          if(action_pos == 3):
            if(state_number % 4 == 0):
              ns = state_number
            else:
              ns = state_number -1 
          if(action_pos == 1):
            if(state_number%4 == 3):
              ns = state_number
            else:
              ns = state_number +1
          if(action_pos == 0 ):
            if(int(state_number / 4) == 0):
              ns = state_number
            else :
              ns = state_number - 4
          if (action_pos == 2):
            if(int(state_number/4)==3):
              ns = state_number
            else:
              ns = state_number +4
          state_prob.append((1,ns,reward,done))
        next_state_probs.append(state_prob)
    return next_state_probs

In [38]:
def policy_evaluation(policy, env, discount_factor = 1 , theta = 0.0001):
  V = np.zeros(env.state_space)
  while True:
    delta = 0
    for s in range(env.state_space):
      v = 0
      for a, action_prob in enumerate(policy[s]):
        prob, next_state, reward, done = env.next_state_probabilities()[s][a]
        v += action_prob * prob * (reward + discount_factor*V[next_state])
      delta = max(delta, np.abs(v - V[s]))
      V[s]= v
    if(delta < theta):
      break
  return V

#env = GridworldEnv()
#policy = np.ones([env.state_space, env.action_space])/env.action_space
#print(policy_evaluation(policy,env))

In [41]:
def policy_improvement(env,discount_factor=1):
  policy = np.ones([env.state_space,env.action_space]) / env.action_space
  while True:
    V = policy_evaluation(policy, env)
    policy_stable = True
    for s in range(env.state_space):
      action_values = np.zeros(env.action_space)
      for a in range(env.action_space):
        prob, next_state , reward , done = env.next_state_probabilities()[s][a]
        action_values[a] += prob * (reward + discount_factor*V[next_state])
      choosen_a = np.argmax(policy[s])
      best_action = np.argmax(action_values)
      if(choosen_a != best_action):
        policy_stable = False
      policy[s] = np.eye(env.action_space)[best_action]
    if(policy_stable == True):
      return policy, V

In [46]:
def value_iteration(env,theta=0.001, discount_factor=1):
  V = np.zeros(env.state_space)
  while True:
    delta = 0
    for s in range(env.state_space):
      A = np.zeros(env.action_space)
      for a in range(env.action_space):
        prob, next_state, reward , done = env.next_state_probabilities()[s][a]
        A[a] += prob * (reward + discount_factor*V[next_state])
      best_action = max(A)
      delta = max(delta, np.abs(best_action - V[s]))
      V[s] = best_action
    if(delta< theta):
      break
  
  policy = np.zeros([env.state_space,env.action_space])
  for s in range(env.state_space):
    A = np.zeros(env.action_space)
    for a in range(env.action_space):
      prob, next_state, reward , done = env.next_state_probabilities()[s][a]
      A[a] += prob * (reward + discount_factor*V[next_state])
    best_a = np.argmax(A)
    policy[s][best_a] = 1
  return policy, V
env = GridworldEnv()
print(value_iteration(env))

(array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]]), array([ 0., -1., -2., -3., -1., -2., -3., -2., -2., -3., -2., -1., -3.,
       -2., -1.,  0.]))
